In [11]:
#Import required libraries
from pandas_datareader import data as pdr
import pandas as pd
import yfinance as yahoo
import math

yahoo.pdr_override()
sd = '2021-05-02' #Start Date
ed = '2021-11-02' #End Date

In [12]:
#Simple function to return the number of indices given a dataframe.
def countindices(df):
    newdf = df[['Open']]
    count = 0
    for i in df.index:
        count += 1
    return count

In [13]:
#Given stock ticker, a time period, and an interval, scrapes data from Yahoo! Finance api and aggregates data 
#as shown below.
def gatherStockInfo(ticker, start_date, end_date, interval):
    df = pdr.get_data_yahoo(ticker,start_date, end_date, interval=interval)
    x = [0]
    i = 0
    #Calculates and aggregates change column
    count = countindices(df)
    while(i < count - 1):
        x.append(df['Close'][i + 1] - df['Close'][i])
        i += 1
    df.insert(5, 'Change', x)
    y = []
    for i in range (15 * 7):
        y.append(0)


    i = (15 * 7)
    length = 14
    #Calculates and aggregates RSI column from formula.
    #It essentially sums up the positive changes and negative changes over a 2 week time period and divides each
    #sum by the time period. It then finds the ratio of these positive and negative changes.
    while(i < count ):

        sumUp = 0
        sumDown = 0
        j = i - length
        while(j < i):
            if(df['Change'][j] >= 0):
                sumUp += df['Change'][j]
            else:
                sumDown += df['Change'][j]
            j += 1

        avgUp = sumUp / length
        avgDown =  -1 * (sumDown / length)

        RS = avgUp / avgDown
        RSI = 100 - (100/(1 + RS))

        y.append(RSI)

        i += 1
    df.insert(6, 'RSI', y)
    return (df)


#Print raw data from Yahoo! Finance, nice and clean data types!
pdr.get_data_yahoo("GOOGL", "2021-02-20", "2022-02-20", "1D")[150:170].head(10)




[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
Date,,,,,,,,
2021-09-24,2810.770020,2846.610107,2804.820068,2844.300049,2844.300049,1149200,0,0
2021-09-27,2816.090088,2838.770020,2801.350098,2821.439941,2821.439941,1094100,0,0
2021-09-28,2773.100098,2781.929932,2707.199951,2716.600098,2716.600098,2288600,0,0
2021-09-29,2739.600098,2743.020020,2677.300049,2687.070068,2687.070068,1542400,0,0
2021-09-30,2685.229980,2710.850098,2671.120117,2673.520020,2673.520020,1899800,0,0
2021-10-01,2688.949951,2738.209961,2681.459961,2730.860107,2730.860107,1768000,0,0
2021-10-04,2719.209961,2719.209961,2621.000000,2673.189941,2673.189941,2560100,0,0
2021-10-05,2680.820068,2744.489990,2680.820068,2720.459961,2720.459961,1620200,0,0
2021-10-06,2695.000000,2755.000000,2691.000000,2751.300049,2751.300049,1218200,0,0


In [14]:
#Must start after row 105 because RSI needs the first 15 periods to begin calculations
#Aggregated Data with RSI and Change
gatherStockInfo("GOOGL", "2021-02-20", "2022-02-20", "1D")[150:170].head(10)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Change,RSI,Volume
Date,,,,,,,,
2021-09-24,2810.770020,2846.610107,2804.820068,2844.300049,2844.300049,19.979980,43.591435,1149200
2021-09-27,2816.090088,2838.770020,2801.350098,2821.439941,2821.439941,-22.860107,45.436868,1094100
2021-09-28,2773.100098,2781.929932,2707.199951,2716.600098,2716.600098,-104.839844,40.730354,2288600
2021-09-29,2739.600098,2743.020020,2677.300049,2687.070068,2687.070068,-29.530029,32.100754,1542400
2021-09-30,2685.229980,2710.850098,2671.120117,2673.520020,2673.520020,-13.550049,30.272077,1899800
2021-10-01,2688.949951,2738.209961,2681.459961,2730.860107,2730.860107,57.340088,33.097022,1768000
2021-10-04,2719.209961,2719.209961,2621.000000,2673.189941,2673.189941,-57.670166,37.252612,2560100
2021-10-05,2680.820068,2744.489990,2680.820068,2720.459961,2720.459961,47.270020,32.496093,1620200
2021-10-06,2695.000000,2755.000000,2691.000000,2751.300049,2751.300049,30.840088,33.745198,1218200


In [15]:
#Simple functions for increased readability and programmers ease, they do as the name suggests.
def dfhead(df):
    df.head()#Prints first 5 rows of dataframe
    
def saveDataFrame(df, file_name):
    df.to_csv(file_name)#Saves dataframe
    
    
    #Finds average of open and close price
def average(df):
    newDF = df[['Open', 'Close']]
    print(newDF.head())
    total = 0
    count = 0
    for i in newDF.index:
        total = total + ((newDF['Close'][i] - newDF['Open'][i]) / newDF['Close'][i])
        count = count + 1

    return(total)

In [16]:
#Function which simulates trading with the strategy. 
#Script "buys shares" when RSI crosses above 30. Sells shares when RSI crosses below 70.
def rsiStrategy(df):
    startingMoney = 5000
    totalMoney = startingMoney
    totalShares = 0
    count = countindices(df)
    i = (15 * 7) + 1
    while(i < count):


        if (df['RSI'][i] > 30) & (df['RSI'][i - 1] < 30):
            if (totalMoney >= df['Close'][i]):
                newShares = (totalMoney / df['Close'][i])
                newShares = math.floor(newShares)
                totalShares += newShares
                totalMoney -= newShares * df['Close'][i]


        if (df['RSI'][i] < 70) & (df['RSI'][i - 1] > 70):
            if (totalShares > 0):
                totalMoney += totalShares * df['Close'][i]
                totalShares = 0

        i += 1

    if(totalShares > 0):
        totalMoney += totalShares * df['Close'][count - 1]
    return(((totalMoney - startingMoney)/ totalMoney) * 100)

In [17]:
#Stocks to perform script 
watchlist = ["^GSPC", "AMC", 'GOOG','PFE', 'PYPL', 'TSLA', 'SHOP', "F", "GE"]

highestROI = 0;
newDF = [['Ticker', 'ROI']]
RSIROIArray = []
ROIArray = []

#Uses RSI strategy.
for i in watchlist:
    df = gatherStockInfo(i, sd, ed, '1h')
    RSIROI = rsiStrategy(df)
    RSIROIArray.append(RSIROI)

#Uses a buy(at beginning of period) and hold strategy regardless of price or indicators
for i in watchlist:
    ROIdf = gatherStockInfo(i, sd, ed, '1h')
    Count = countindices(df)
    ROI = (ROIdf['Close'][Count - 1] - ROIdf['Close'][0]) / (ROIdf['Close'][Count - 1]) * 100
    ROIArray.append(ROI)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [20]:
#Creates new dataFrame to present results of study
data = {'Ticker': watchlist, 'RSIROI': RSIROIArray, 'ROI': ROIArray}
newdf = pd.DataFrame(data)

saveDataFrame(newdf, '6-Month-ROI.csv')

newdf.head() #Data frame resulting in the Return-On-Investment of RSI Strategy(Left) and Hold Strategy(Right)
#Further proves the saying, "Time in the market beats timing the market".

,Ticker,RSIROI,ROI
0,^GSPC,5.464895,8.868874
1,AMC,-18.133985,73.151647
2,GOOG,7.671735,15.999741
3,PFE,-3.111757,9.798307
4,PYPL,0.111082,-14.196175
